# Analyzing Big Data

Big data means collecting, storing, and processing data at a large scale on multiple machines.

## The Challenges of Data Science

1. Vast majority of work goes into feature engineering and selection, i.e preprocessing steps.
2. Iteration: Modeling and analysis require multiple passes over the same data.
3. Provide the same model in data applications.

## Introducing Apache Spark

Apache Spark is an open source framework that combines an engine for distributing programs across clusters of machines with an elegant model for writing programs atop it.

Features:

1. Rather than relying on a rigid map-then-reduce format, its engine can execute a more general directed acyclic graph (DAG) of operators.
2. it complements this capability with a rich set of transformations that enable users to express computation more naturally.
3. Spark extends its predecessors with in-memory processing.
4. Sitting atop the JVM, it can take advantage of many of the operational and debugging tools built for the Java stack.
5. Spark boasts strong integration with the variety of tools in the Hadoop ecosystem.